# Local Learning for benchmark models
This includes: 
- **Dense**: 3 layers, with 16 units and relu activation function
- **Bidirectional LSTM**: 2 bidirectional layers, with 20 LSTM cells and tanh 

In [1]:
#Imports
import pandas as pd
import os
import tensorflow as tf

from utils.modelgenerator import *
from utils.modelhandler import *
from utils.datahandler import *

#Get data 
cwd = os.path.normpath(os.getcwd())
df = pd.read_csv(cwd+'/data/df_with_final_features.csv', index_col='Date') #df = pd.read_csv('user5.csv')
df.index = pd.to_datetime(df.index)
#df = df[['User5', 'temp', 'rhum']]
df.fillna(0, inplace=True)

df_array = []
for idx in range(30):
    df_array.append(df[[f'User{idx+1}', 'temp', 'rhum', 'wspd', 'PC1', 'hour sin', 'hour cos', f'User{idx+1}_lag_24hrs']])

#df_array[3].head(3)

#Train, Validation and Test datasets
sequence_length = 25
batch_size = 16
num_features = df_array[0].shape[1]

dh = Datahandler()

X_train, y_train, X_val, y_val, X_test, y_test = {}, {}, {}, {}, {}, {}

#Create Train, Validation and Test datasets
for idx, df in enumerate(df_array):
    n = len(df)
    train_df = df[0:int(n*0.7)]
    val_df = df[int(n*0.7):int(n*0.9)]
    test_df = df[int(n*0.9):]

    # Min max sclaing
    train_df = dh.min_max_scaling(train_df)
    val_df = dh.min_max_scaling(val_df)
    test_df = dh.min_max_scaling(test_df)

    # Sequencing
    train_sequences = dh.create_sequences(train_df, sequence_length)
    val_sequences = dh.create_sequences(val_df, sequence_length)
    test_sequences = dh.create_sequences(test_df, sequence_length)

    #Split into feature and label
    X_train[f'user{idx+1}'], y_train[f'user{idx+1}'] = dh.prepare_data(train_sequences, batch_size)
    X_val[f'user{idx+1}'], y_val[f'user{idx+1}'] = dh.prepare_data(val_sequences, batch_size)
    X_test[f'user{idx+1}'], y_test[f'user{idx+1}'] = dh.prepare_data(test_sequences, batch_size)

#General Hyperparameters
# #All models
horizon = 1
max_epochs = 100
m1 = ModelGenerator()
mh = Modelhandler()

loss = tf.keras.losses.MeanSquaredError()
metrics=[
    tf.keras.metrics.RootMeanSquaredError(), 
    tf.keras.metrics.MeanAbsolutePercentageError(),
    tf.keras.metrics.MeanAbsoluteError(),
]

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,mode='min')
timing_callback = TimingCallback()
custom_callback = CustomCallback()
#model_checkpoint = ModelCheckpoint('models/best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
callbacks=[early_stopping, timing_callback, custom_callback] #model_checkpoint


# 1. SOFT DENSE MOE MODEL

In [2]:
dense_results = pd.DataFrame(columns=['architecture', 'train_time', 'avg_time_epoch', 'mse','mse_std', 'rmse','rmse_std','mape','mape_std','mae','mae_std'])

In [3]:
#MOE MODEL 1
dense_moe_architecture = "Soft_Dense_MoE"
dense_units = 16
num_experts = 4
expert_units = 8

dense_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])

#For each of the 3 user
for idx in range(len(df_array)):
    print("User: ", idx+1)
    for round in range(3):
        #print("Round: ", round)
        dense_model = m1.build_soft_dense_moe_model(X_train[f'user{idx+1}'], batch_size, horizon, dense_units, expert_units, num_experts, m1)
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=dense_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=dense_moe_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
        )
        # Add the 'architecture' column from dense_user_results to dense_results
        dense_all_results = pd.merge(dense_all_results, dense_user_results, how='outer')   

    dense_model.save(cwd + f"/models/Local_learning/{dense_moe_architecture}/User{idx}")
    print("Saved Soft_Dense_MoE")  



for idx in range(len(df_array)):
    new_row = {
        'architecture': dense_all_results["architecture"][0],
        'train_time': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    dense_results.loc[len(dense_results)] = new_row

User:  1
INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/Soft_Dense_MoE/User0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/Soft_Dense_MoE/User0\assets


Saved Soft_Dense_MoE
User:  2
INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/Soft_Dense_MoE/User1\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/Soft_Dense_MoE/User1\assets


Saved Soft_Dense_MoE
User:  3
INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/Soft_Dense_MoE/User2\assets


...

Saved Soft_Dense_MoE
User:  29
INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/Soft_Dense_MoE/User28\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/Soft_Dense_MoE/User28\assets


Saved Soft_Dense_MoE
User:  30
INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/Soft_Dense_MoE/User29\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/Soft_Dense_MoE/User29\assets


Saved Soft_Dense_MoE


In [4]:
dense_results.to_csv(f'evaluations/Local_learning/{dense_moe_architecture}.csv')
dense_results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Soft_Dense_MoE,15.626359,0.463006,0.034008,0.001083,0.184397,0.002948,8.830589e+04,19232.836431,0.129079,0.003706
1,Soft_Dense_MoE,9.322736,0.478693,0.018306,0.000799,0.135277,0.002945,1.604002e+05,29655.439462,0.076615,0.001233
2,Soft_Dense_MoE,11.822861,0.475102,0.019897,0.000272,0.141054,0.000964,5.190807e+04,10837.389705,0.079009,0.003142
3,Soft_Dense_MoE,13.700077,0.461089,0.023119,0.001371,0.152004,0.004542,1.215700e+05,16326.066329,0.065597,0.003234
4,Soft_Dense_MoE,15.611840,0.485246,0.014659,0.000378,0.121067,0.001568,9.773217e+04,21146.666917,0.079981,0.007227
5,Soft_Dense_MoE,10.895442,0.484592,0.019896,0.000916,0.141030,0.003240,9.753535e+04,27478.223929,0.101038,0.003678
6,Soft_Dense_MoE,9.237150,0.481440,0.020026,0.000957,0.141486,0.003378,9.437673e+04,5333.592766,0.080616,0.002861
7,Soft_Dense_MoE,15.180292,0.477294,0.023913,0.001063,0.154613,0.003416,2.596664e+04,19514.097835,0.102416,0.001870
8,Soft_Dense_MoE,8.213238,0.521750,0.030319,0.003107,0.173967,0.009066,7.344347e+01,5.273183,0.103316,0.004435
9,Soft_Dense_MoE,13.226991,0.494499,0.014332,0.002040,0.119520,0.008435,2.289266e+06,222486.286993,0.072292,0.008905


# Model top k Dense MoE

In [5]:
dense_moe_architecture = "TopK_Dense_MoE" #top2_exp5_d8
dense_units = 16

num_experts = 5
expert_units = 8
top_k = 2

dense_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])
#For each of the 3 user
for idx in range(len(df_array)):
    print("User: ", idx+1)
    for round in range(3):
        #print("Round: ", round)
        dense_model = m1.build_topk_dense_moe_model(X_train[f'user{idx+1}'], batch_size, horizon, dense_units, num_experts, top_k, expert_units, m1)
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=dense_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=dense_moe_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
        )
        # Add the 'architecture' column from dense_user_results to dense_results
        dense_all_results = pd.merge(dense_all_results, dense_user_results, how='outer')   

    dense_model.save(cwd + f"/models/Local_learning/{dense_moe_architecture}/User{idx}")
    print("Saved Soft_Dense_MoE")  

for idx in range(len(df_array)):
    new_row = {
        'architecture': dense_all_results["architecture"][0],
        'train_time': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    dense_results.loc[len(dense_results)] = new_row

User:  1
INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/TopK_Dense_MoE/User0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/TopK_Dense_MoE/User0\assets


Saved Soft_Dense_MoE
User:  2
INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/TopK_Dense_MoE/User1\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/TopK_Dense_MoE/User1\assets


Saved Soft_Dense_MoE
User:  3
INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/TopK_Dense_MoE/User2\assets


...

Saved Soft_Dense_MoE
User:  29
INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/TopK_Dense_MoE/User28\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/TopK_Dense_MoE/User28\assets


Saved Soft_Dense_MoE
User:  30
INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/TopK_Dense_MoE/User29\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/TopK_Dense_MoE/User29\assets


Saved Soft_Dense_MoE


In [6]:
dense_results.to_csv(f'evaluations/Local_learning/{dense_moe_architecture}.csv')
dense_results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Soft_Dense_MoE,15.626359,0.463006,0.034008,0.001083,0.184397,0.002948,8.830589e+04,19232.836431,0.129079,0.003706
1,Soft_Dense_MoE,9.322736,0.478693,0.018306,0.000799,0.135277,0.002945,1.604002e+05,29655.439462,0.076615,0.001233
2,Soft_Dense_MoE,11.822861,0.475102,0.019897,0.000272,0.141054,0.000964,5.190807e+04,10837.389705,0.079009,0.003142
3,Soft_Dense_MoE,13.700077,0.461089,0.023119,0.001371,0.152004,0.004542,1.215700e+05,16326.066329,0.065597,0.003234
4,Soft_Dense_MoE,15.611840,0.485246,0.014659,0.000378,0.121067,0.001568,9.773217e+04,21146.666917,0.079981,0.007227
5,Soft_Dense_MoE,10.895442,0.484592,0.019896,0.000916,0.141030,0.003240,9.753535e+04,27478.223929,0.101038,0.003678
6,Soft_Dense_MoE,9.237150,0.481440,0.020026,0.000957,0.141486,0.003378,9.437673e+04,5333.592766,0.080616,0.002861
7,Soft_Dense_MoE,15.180292,0.477294,0.023913,0.001063,0.154613,0.003416,2.596664e+04,19514.097835,0.102416,0.001870
8,Soft_Dense_MoE,8.213238,0.521750,0.030319,0.003107,0.173967,0.009066,7.344347e+01,5.273183,0.103316,0.004435
9,Soft_Dense_MoE,13.226991,0.494499,0.014332,0.002040,0.119520,0.008435,2.289266e+06,222486.286993,0.072292,0.008905


# MOE Bidirectional LSTM Model

In [7]:
bilstm_moe_architecture = "Soft_BiLSTM_MoE"
lstm_units = 4
num_experts = 4
expert_units = 8

dense_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])
#For each of the 3 user
for idx in range(len(df_array)):
    print("User: ", idx+1)
    for round in range(3):
        #print("Round: ", round)
        dense_model = m1.build_soft_biLSTM_moe_model(X_train[f'user{idx+1}'], batch_size, horizon, lstm_units, num_experts, expert_units, m1)
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=dense_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=dense_moe_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
        )
        # Add the 'architecture' column from dense_user_results to dense_results
        dense_all_results = pd.merge(dense_all_results, dense_user_results, how='outer')   

    dense_model.save(cwd + f"/models/Local_learning/{bilstm_moe_architecture}/User{idx}")
    print("Saved Soft_Dense_MoE")  

for idx in range(len(df_array)):
    new_row = {
        'architecture': dense_all_results["architecture"][0],
        'train_time': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    dense_results.loc[len(dense_results)] = new_row

User:  1


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/Soft_BiLSTM_MoE/User0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/Soft_BiLSTM_MoE/User0\assets


Saved Soft_Dense_MoE
User:  2
INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/Soft_BiLSTM_MoE/User1\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/Soft_BiLSTM_MoE/User1\assets


...

Saved Soft_Dense_MoE
User:  29
INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/Soft_BiLSTM_MoE/User28\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/Soft_BiLSTM_MoE/User28\assets


Saved Soft_Dense_MoE
User:  30
INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/Soft_BiLSTM_MoE/User29\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/Soft_BiLSTM_MoE/User29\assets


Saved Soft_Dense_MoE


In [8]:
dense_results.to_csv(f'evaluations/Local_learning/{bilstm_moe_architecture}.csv')
dense_results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Soft_Dense_MoE,15.626359,0.463006,0.034008,0.001083,0.184397,0.002948,8.830589e+04,19232.836431,0.129079,0.003706
1,Soft_Dense_MoE,9.322736,0.478693,0.018306,0.000799,0.135277,0.002945,1.604002e+05,29655.439462,0.076615,0.001233
2,Soft_Dense_MoE,11.822861,0.475102,0.019897,0.000272,0.141054,0.000964,5.190807e+04,10837.389705,0.079009,0.003142
3,Soft_Dense_MoE,13.700077,0.461089,0.023119,0.001371,0.152004,0.004542,1.215700e+05,16326.066329,0.065597,0.003234
4,Soft_Dense_MoE,15.611840,0.485246,0.014659,0.000378,0.121067,0.001568,9.773217e+04,21146.666917,0.079981,0.007227
...,...,...,...,...,...,...,...,...,...,...,...
85,TopK_Dense_MoE,53.280330,1.398305,0.023779,0.001476,0.154155,0.004825,1.363704e+05,25981.224075,0.091218,0.001862
86,TopK_Dense_MoE,50.668634,1.393632,0.017601,0.000571,0.132656,0.002160,2.413912e+05,13535.949480,0.088671,0.000721
87,TopK_Dense_MoE,36.918722,1.415364,0.010855,0.000199,0.104184,0.000952,6.014136e+04,5915.740136,0.045907,0.002740
88,TopK_Dense_MoE,47.375805,1.389546,0.016744,0.000276,0.129397,0.001070,3.555199e+05,44652.313194,0.090446,0.000722


# Top K MoE Bidirectional LSTM

In [9]:
bilstm_moe_architecture = "TopK_BiLSTM_MoE"
lstm_units = 8

num_experts = 4
top_k = 2
expert_units = 8

dense_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])
#For each of the 3 user
for idx in range(len(df_array)):
    print("User: ", idx+1)
    for round in range(3):
        #print("Round: ", round)
        dense_model = m1.build_topk_bilstm_moe_model(X_train[f'user{idx+1}'], batch_size, horizon, lstm_units, num_experts, top_k, expert_units, m1)
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=dense_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=dense_moe_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
        )
        # Add the 'architecture' column from dense_user_results to dense_results
        dense_all_results = pd.merge(dense_all_results, dense_user_results, how='outer')   

    dense_model.save(cwd + f"/models/Local_learning/{bilstm_moe_architecture}/User{idx}")
    print("Saved Soft_Dense_MoE")  


for idx in range(len(df_array)):
    new_row = {
        'architecture': dense_all_results["architecture"][0],
        'train_time': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    dense_results.loc[len(dense_results)] = new_row

User:  1
INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/TopK_BiLSTM_MoE/User0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/TopK_BiLSTM_MoE/User0\assets


Saved Soft_Dense_MoE
User:  2
INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/TopK_BiLSTM_MoE/User1\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/TopK_BiLSTM_MoE/User1\assets


Saved Soft_Dense_MoE
User:  3
INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/TopK_BiLSTM_MoE/User2\assets


...

Saved Soft_Dense_MoE
User:  29
INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/TopK_BiLSTM_MoE/User28\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/TopK_BiLSTM_MoE/User28\assets


Saved Soft_Dense_MoE
User:  30
INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/TopK_BiLSTM_MoE/User29\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/TopK_BiLSTM_MoE/User29\assets


Saved Soft_Dense_MoE


In [10]:
dense_results.to_csv(f'evaluations/Local_learning/{bilstm_moe_architecture}.csv')
dense_results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Soft_Dense_MoE,15.626359,0.463006,0.034008,0.001083,0.184397,0.002948,8.830589e+04,19232.836431,0.129079,0.003706
1,Soft_Dense_MoE,9.322736,0.478693,0.018306,0.000799,0.135277,0.002945,1.604002e+05,29655.439462,0.076615,0.001233
2,Soft_Dense_MoE,11.822861,0.475102,0.019897,0.000272,0.141054,0.000964,5.190807e+04,10837.389705,0.079009,0.003142
3,Soft_Dense_MoE,13.700077,0.461089,0.023119,0.001371,0.152004,0.004542,1.215700e+05,16326.066329,0.065597,0.003234
4,Soft_Dense_MoE,15.611840,0.485246,0.014659,0.000378,0.121067,0.001568,9.773217e+04,21146.666917,0.079981,0.007227
...,...,...,...,...,...,...,...,...,...,...,...
115,TopK_Dense_MoE,67.585376,1.725003,0.023029,0.000502,0.151748,0.001649,9.423817e+04,41452.526190,0.095187,0.002600
116,TopK_Dense_MoE,51.862859,1.728453,0.019542,0.000670,0.139778,0.002393,2.625060e+05,63509.833296,0.099187,0.006676
117,TopK_Dense_MoE,38.906657,1.733680,0.011191,0.000381,0.105778,0.001795,5.523744e+04,12154.412663,0.045857,0.001533
118,TopK_Dense_MoE,55.663867,1.731980,0.017417,0.000782,0.131952,0.002964,4.316151e+05,69795.310530,0.092368,0.000741


In [14]:
print("Soft MoE Dense: ", dense_results[0:29]["mse"].mean())
print("TopK MoE Dense: ", dense_results[30:59]["mse"].mean())
print("Soft MoE Dense: ", dense_results[60:89]["mse"].mean())
print("Soft MoE Dense: ", dense_results[90:119]["mse"].mean())

Soft MoE Dense:  0.01844200170224254
TopK MoE Dense:  0.018917902565464887
Soft MoE Dense:  0.017549493125286596
Soft MoE Dense:  0.01822167591074075
